# Deep Learning 1

In this practical session, we will take a look at the [Keras](https://keras.io/) deep learning framework for Python. From the documentation:

>Deep learning for humans. Keras is an API designed for human beings, not machines. Keras follows best practices for reducing cognitive load: it offers consistent & simple APIs, it minimizes the number of user actions required for common use cases, and it provides clear & actionable error messages. It also has extensive documentation and developer guides.

Designing and testing neural networks can be a tedious process. Keras provides an intuitive API for training, testing and deploying neural networks without having to worry too much about technical details. To begin, we import TensorFlow:

In [ ]:
import tensorflow as tf

Keras comes bundled with TensorFlow 2 under the `tensorflow.keras` module, which we will be using in the sequel.

## Deep Neural Network basics

A Deep Neural Network (DNN) is built up of several layers. Formally, such a network is given by

$$
    f(x) = g_L(W_Lg_{L-1}(\dots g_1(W_1x + b_1) \dots) + b_L)
$$

Here, $W_1, \dots, W_L$ are matrices (called the *weights* of the network), $b_1, \dots, b_L$ are vectors (called the *biases*) and $g_1, \dots, g_L$ are the *activation functions*. We can picture this as follows:

![A neural network](mlp.png)

Each node in the network computes an inner product plus a bias term between its weight vector, which is a row of the weight matrix of that layer, and its input, which is the output of the previous layer. This inner product is then sent through a non-linear activation function. Examples of typical activation functions are

1. the logistic sigmoid:
$$
    \mathrm{sigmoid}(z) = \frac{1}{1 + \exp(-z)},
$$

2. the rectified linear unit (RELU):
$$
    \mathrm{relu}(z) = \max(0,z),
$$

3. the scaled exponential linear unit (SELU):
$$
    \mathrm{selu}(z) = \lambda\left\{\begin{matrix}
        z, & \mbox{if $z > 0$}\\
        \alpha(\exp(z)-1), & \mbox{if $z \leq 0$}
    \end{matrix}\right..
$$
Here, $\lambda$ and $\alpha$ are parameters fixed before the network is trained.

4. the softmax:
$$
    \mathrm{softmax}(z) = \frac{\exp(z)}{\sum_i\exp(z_i)}.
$$
The softmax function is only used at the very end of a neural network classifier. It has the properties that for any $z \in \mathbb{R}^q$,
$$\begin{aligned}
    \mathrm{softmax}(z) &\in [0,1]^q, & \sum_{i=1}^q\mathrm{softmax}(z)_i &= 1.
\end{aligned}$$
In other words, the output of $\mathrm{softmax}(z)$ can be interpreted as a vector of probabilities over $q$ possible outputs.

We can construct a simple neural network in Keras as follows:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=256, activation='relu', input_dim=7),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

This model consists of three *dense* or *fully-connected* layers. These are the "classical" neural network layers which perform a linear transformation and then apply a specified activation function. In our case, the full network is mathematically given by

$$
    f: \mathbb{R}^{7} \to \mathbb{R}^{2}: x \mapsto \mathrm{softmax}(W_3\mathrm{relu}(W_2\mathrm{relu}(W_1x + b_1) + b_2) + b_3).
$$

Moreover, $W_1 \in \mathbb{R}^{256 \times 7}$, $W_2 \in \mathbb{R}^{256 \times 256}$, $W_3 \in \mathbb{R}^{1 \times 256}$, $b_1 \in \mathbb{R}^{256}$, $b_2 \in \mathbb{R}^{256}$ and $b_3 \in \mathbb{R}$. In all, this model has

$$
    256 \times 7 + 256 \times 256 + 256 \times 1 + 256 + 256 + 1 = 68 097
$$

parameters. Keras can also confirm this:

In [ ]:
print(model.summary())

Of course, the weight matrices and bias vectors have not been set to any meaningful values yet. Finding values for these parameters such that the network maximizes a certain performance measure is the goal of a *learning algorithm*.

## Supervised learning

In the supervised learning setting, we are given a data set of observations,

$$
    D = \{ (x_i,y_i) \in \mathbb{R}^d \times \{1, \dots, C\} \mid i = 1, \dots, N \}.
$$

These observations are assumed to be independent and identically distributed according to a probability measure $P$. Each element $(x_i,y_i) \in D$ consists of a raw input vector $x_i \in \mathbb{R}^d$ and a ground-truth label $y_i \in \{1, \dots, C\}$. Our goal is to build a neural network $f(x)$ such that

$$
    \Pr_{(x,y) \sim P}[f(x) = y] = 1.
$$

Of course, we don't know the measure $P$, otherwise there would be no learning problem. So instead, we aim to minimize an *empirical risk function*

$$
    R_{\mathrm{emp}}(f) = \frac{1}{N}\sum_{i=1}^N\ell(f(x_i), y_i).
$$

This is an empirical approximation to the *expected risk*

$$
    R(f) = \mathrm{E}_{(x,y) \sim P}[\ell(f(x),y)],
$$

which we cannot compute. Here, $\ell(f(x),y)$ is the *loss function* which measures how much the output $f(x)$ deviates from the desired output $y$. In a classification problem, usually the 0/1 loss is used:

$$
    \ell(f(x),y) = \left\{\begin{matrix}
        1, & \mbox{if $f(x) \neq y$}\\
        0, & \mbox{if $f(x) = y$}
    \end{matrix}\right..
$$

However, in some cases (e.g. regression problems) it makes sense to use other functions such as the squared error

$$
    \ell(f(x),y) = \|f(x)-y\|_2^2.
$$

We now need to solve the optimization problem

$$
    \min_f R_{\mathrm{emp}}(f).
$$

In practice, this means adjusting our weight matrices and bias vectors until we don't get anymore reductions in empirical risk. There exist a variety of optimization algorithms to solve this problem, but we won't get into their details here. What is important to know is that all of these optimization algorithms work iteratively on *mini-batches* of the training data set, never on the entire training set at once unless it is really small (which it often isn't). The number of mini-batches is determined by the *batch size*, which is the size of a mini-batch. So the optimization algorithms proceed as follows:

1. Split the training set into mini-batches with a number of samples equal to the batch size.
2. For each mini-batch, compute updates to the weights and biases based solely on the given mini-batch data.
3. Repeat step 2 until convergence.

Each run of the loop in step 2 is called an *epoch*. You will always need to specify the number of epochs as well as the batch size whenever you train a neural network. Make sure that the batch size is always a divisor of the training set size, otherwise some algorithms may refuse to run. Typical batch sizes are small powers of two such as 64 or 128. In case the training set size happens to be a prime number (which can occur e.g. if you augment the training set with artificial samples), you may need to subsample.

To train our model in Keras, we compile it with a loss function and an optimizer which will attempt to minimize the loss:

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

Note that the `metrics` takes a list as an argument, because multiple metrics can be computed for the model. See [the documentation](https://keras.io/metrics/) for an exhaustive list of supported metrics. The model can now be fit to a data set. We will try the Titanic data set again:

In [ ]:
import numpy as np
from sklearn import preprocessing

X = np.loadtxt("titanic.csv", delimiter=",")
y = np.loadtxt("labels.csv", delimiter=",").astype(np.int)
X = preprocessing.scale(X)

We now shuffle and split the data set into training and test sets:

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
train_idx, test_idx = next(sss.split(X, y))

x_train, y_train = X[train_idx], y[train_idx]
x_test, y_test = X[test_idx], y[test_idx]

Note that we have z-normalized the samples to zero mean and unit variance. Now it's up to Keras to fit the model to this data set:

In [ ]:
model.fit(x_train, y_train, epochs=20, batch_size=10, validation_split=.2)

Due to randomness in the initialization of the weights and biases as well as the optimization algorithm itself, repeated runs of the `fit` method will yield different results. You should be able to get at least 80% test accuracy after a few runs.

Having trained the model, we can ask it for predictions:

In [ ]:
threshold = .5
classes = (model.predict(x_test, batch_size=64) > threshold).astype(np.int)

Using this array, we can compute the accuracy ourselves:

In [ ]:
accuracy = np.mean(np.equal(classes.flatten(), y_test))
print('Accuracy: {}'.format(accuracy))

Or we could let Keras compute it for us:

In [ ]:
evals = model.evaluate(x_test, y_test, batch_size=65)
print('Loss: {}'.format(evals[0]))
print('Accuracy: {}'.format(evals[1]))

The classes of the Titanic data set are not balanced, so we may want to check out the balanced accuracy as well:

In [ ]:
idx0 = (y_test == 0)
idx1 = (y_test == 1)

acc0 = np.mean(np.equal(classes[idx0], y_test[idx0]))
acc1 = np.mean(np.equal(classes[idx1], y_test[idx1]))
bal_acc = (acc0 + acc1) / 2
print('Balanced accuracy: {}'.format(bal_acc))
print('\tClass 0: {}'.format(acc0))
print('\tClass 1: {}'.format(acc1))

## Exercise 1

Keras supports a number of different loss functions (see [the documentation](https://keras.io/losses/)). Try modifying the loss function we used in the example to something else and retrain the network. Why would you choose a certain loss over another? In particular, why is the 0/1 loss not even on this list?

## Exercise 2

Keras has a number of activation functions built in besides the RELU (see [the documentation](https://keras.io/activations/)). Change the activation functions from RELU to something else and retrain the network. Can you explain the effects of this choice? **Hint:** try visualizing the activation functions with a plot.

## Exercise 3

Compare the performance of the neural network to more "classical" machine learning algorithms such as KNN or support vector machines. Can you get the network to outperform them all?

## Exercise 4

Keras provides a [number of data sets](https://keras.io/datasets/) for you to experiment with. Try training a neural network on the Boston housing price regression data set. Note that this is a *regression problem*, not a *classification problem* as we have considered up until this point. You'll have to change your approach slightly to tackle this problem.